In [1]:
import re

In [2]:
with open("input", "r") as f:
    lines = [tuple(line.strip().split(": ")) for line in f.readlines()]

In [3]:
p = re.compile("Each ([a-z]+) robot costs (\d+) ore(?: and (\d+) (clay|obsidian))?")

In [4]:
_blueprints = {int(bid.split()[1]): [p.match(blueprint).groups()
                                     for blueprint in _blueprint.split('. ')]
               for bid, _blueprint in lines}

In [5]:
blueprints = {}
for i, blueprint in _blueprints.items():
    resources = {}
    for rob_typ, ore, other_quantity, other in blueprint:
        resources[rob_typ] = {'ore': int(ore)}
        if other is not None:
            resources[rob_typ][other] = int(other_quantity)
    blueprints[i] = resources

In [6]:
def has_resources(robot_cost, resources):
    return all(resources[typ] >= robot_cost[typ] for typ in robot_cost.keys())

In [7]:
def is_needed(robot_type, resources, robots):
    if robot_type == 'geode':
        return True
    if robot_type == 'obsidian':
        return (robots['obsidian'] < robot_costs['geode']['obsidian']
                and resources['obsidian'] <= 1 * robot_costs['geode']['obsidian'])
    if robot_type == 'ore':
        return (robots['ore'] < max(robot_costs[typ]['ore'] for typ in robot_costs.keys())
                and any(resources['ore'] <= 1.35 * robot_costs[typ]['ore'] for typ in robot_costs.keys()))
    if robot_type == 'clay':
        return (robots['clay'] < robot_costs['obsidian']['clay']
                and robots['obsidian'] < robot_costs['geode']['obsidian'])
    return False

In [8]:
def geode(t, robot_type, resources, robots):
    
    cache_key = tuple(robots.values()) + tuple(resources.values()) + (robot_type,)
    if cache_key in cache:
        if t >= cache[cache_key]:
            return 0, (robots, resources)
        
    cache[cache_key] = t
    
    _resources = resources.copy()
    _robots = robots.copy()
    
    if t == n_periods - 1:
        for typ in _robots.keys():
            _resources[typ] += _robots[typ]
        
        return (_resources['geode'], (_robots, _resources))
    
    new_robots = {k: 0 for k in _robots.keys()}
    
    if robot_type in robot_costs.keys() and has_resources(robot_costs[robot_type], _resources):
            for resource_type in robot_costs[robot_type].keys():
                assert _resources[resource_type] >= robot_costs[robot_type][resource_type]
                _resources[resource_type] -= robot_costs[robot_type][resource_type]
            new_robots[robot_type] = 1
    
    for typ in _robots.keys():
        _resources[typ] += _robots[typ]
        _robots[typ] += new_robots[typ]
    
    to_build = list(robot_costs.keys()) + ['wait']
    
    if has_resources(robot_costs['geode'], _resources):
        return geode(t + 1, 'geode', _resources, _robots)
    
    return max((geode(t + 1, _robot_type, _resources, _robots)
               for _robot_type in to_build
               if (_robot_type == 'wait'
                   or (has_resources(robot_costs[_robot_type], _resources)
                       and is_needed(_robot_type, _resources, _robots)))), key=lambda x: x[0])

### Part 1

In [9]:
results = []

for i in blueprints.keys():
    print(f"Starting blueprint {i}")
    robots = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
    resources = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}

    robot_costs = blueprints[i]
    cache = {}
    n_periods = 24
    
    results.append((i, geode(0, "wait", resources, robots)))

Starting blueprint 1
Starting blueprint 2
Starting blueprint 3
Starting blueprint 4
Starting blueprint 5
Starting blueprint 6
Starting blueprint 7
Starting blueprint 8
Starting blueprint 9
Starting blueprint 10
Starting blueprint 11
Starting blueprint 12
Starting blueprint 13
Starting blueprint 14
Starting blueprint 15
Starting blueprint 16
Starting blueprint 17
Starting blueprint 18
Starting blueprint 19
Starting blueprint 20
Starting blueprint 21
Starting blueprint 22
Starting blueprint 23
Starting blueprint 24
Starting blueprint 25
Starting blueprint 26
Starting blueprint 27
Starting blueprint 28
Starting blueprint 29
Starting blueprint 30
CPU times: user 3min 42s, sys: 3.95 s, total: 3min 46s
Wall time: 3min 48s


In [10]:
sum(i * res[0] for i, res in results)

1599

### Part 2

In [11]:
results_part2 = []

for i in range(1, 4):
    print(f"Starting blueprint {i}")
    robots = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
    resources = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}

    robot_costs = blueprints[i]
    n_periods = 32
    
    cache = {}
    results_part2.append((i, geode(0, "wait", resources, robots)))

Starting blueprint 1
Starting blueprint 2
Starting blueprint 3


In [12]:
score = 1
for i, (result, _) in results_part2:
    score *= result
score

14112